In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from CoolProp.CoolProp import PropsSI
from pint import UnitRegistry
u = UnitRegistry()


# Options for pandas
#pd.options.display.max_columns = 50
#pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()
from IPython.display import display_latex

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import matplotlib.pyplot as plt
h = 8
w = h*1.61803398875
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 26}

plt.rc('font', **font)

#from matplotlib import rc
plt.rc('font',**{'family':'serif','serif':['Times New Roman']})
plt.rc('text', usetex=True)

# Publish
from ipypublish.scripts import nb_setup

ModuleNotFoundError: No module named 'scipy'

In [ ]:
def chem_prop(pressure, chemical, state):
    '''
    Returns the entropy, specific gravity, enthalpy, and internal energy of a specific chemical
    at saturation.
    '''
    s_g = PropsSI('S','P',pressure,'Q',1,chemical)/1000;
    s_f = PropsSI('S','P',pressure,'Q',0,chemical)/1000; 
    v_g = 1/PropsSI('D','P',pressure,'Q',1,chemical); 
    v_f = 1/PropsSI('D','P',pressure,'Q',0,chemical); 
    h_g = PropsSI('H','P',pressure,'Q',1,chemical); 
    h_f = PropsSI('H','P',pressure,'Q',0,chemical);
    u_g = (h_g - pressure*(v_g))/1000;
    u_f = (h_f - pressure*(v_f))/1000;
    return [chemical, state, pressure, s_g, s_f, v_g, v_f, h_g, h_f, u_g, u_f]

def build_state_table(state1,state2):
    '''
    Returns a dictionary of a chemical at two different states.
    '''
    data = {'State': [state1[0] + '-' + state1[1], state2[0] + '-' + state2[1]],
            '$P \\left(kPa\\right)$': [state1[2]/1000, state2[2]/1000],
            r'$u_f \left(\frac{kJ}{kg\:K}\right)$': [state1[10], state2[10]],
            r'$u_g \left(\frac{kJ}{kg\:K}\right)$': [state1[9], state2[9]],
            r'$v_f \left(\frac{m^3}{kg}\right)$': [state1[6], state2[6]],
            r'$v_g \left(\frac{m^3}{kg}\right)$': [state1[5], state2[5]],
            r'$s_f \left(\frac{kJ}{kg\:K}\right)$': [state1[4], state2[4]],
            r'$s_g \left(\frac{kJ}{kg\:K}\right)$': [state1[3], state2[3]],
            r'$h_f \left(\frac{kJ}{kg}\right)$': [state1[8]/1000, state2[8]/1000],
            r'$h_g \left(\frac{kJ}{kg}\right)$': [state1[7]/1000, state2[7]/1000]}
    return data

def ellipsoidal_head(diameter, thickness):
    '''
    Returns the volume (lb) of material in an ellispoidal head (2:1) given the
    diameter (arg1) and thickness (arg2).
    '''
    o_dia = diameter
    i_dia = diameter - 2*thickness
    ellipsoidal_head_outer = ((np.pi)*(o_dia**2)*(o_dia/4))/6
    ellipsoidal_head_inner = ((np.pi)*(i_dia**2)*(i_dia/4))/6
    volume_of_head = ellipsoidal_head_outer - ellipsoidal_head_inner
    return volume_of_head



def cylinder(length, diameter, thickness):
    '''
    Returns the volume of material in a cylinder given the length (arg1),
    diameter (arg2), thickness (arg3).
    '''
    o_dia = diameter
    i_dia = diameter - 2*thickness
    cylinder_outer = ((np.pi)*(o_dia**2)*(length))/4
    cylinder_inner = ((np.pi)*(i_dia**2)*(length))/4
    volume_of_cylinder = cylinder_outer - cylinder_inner
    return volume_of_cylinder

def print_ie(state,chemical,row):
    '''
    Prints the chemical (arg2) u = h - pv for each state (arg1) using the dataframe row (arg3)
    '''
    #chemical
    #1
    #fluid
    hf = df.iloc[row][7]
    uf = df.iloc[row][1]
    p = df.iloc[row][0]
    vf = df.iloc[row][3]
    #gas
    hg = df.iloc[row][8]
    ug = df.iloc[row][2]
    p = df.iloc[row][0]
    vg = df.iloc[row][4]
    #print
    print('fluid %s state %s: %8.2f = %8.2f - (%8.2f)(%8.6f)' % (chemical,state,uf,hf,p,vf))
    print('gaseous %s state %s: %8.2f = %8.2f - (%8.2f)(%8.6f)' % (chemical,state,ug,hg,p,vg))
    return

def quality(chem_state1, chem_state2):
    s_f1 = chem_state1[4]
    s_f2 = chem_state2[4]
    s_g1 = chem_state1[3]
    s_g2 = chem_state2[3]
    
    Xf = (s_f1 - s_f2)/(s_g2-s_f2)
    Xg = (s_g2 - s_g1)/(s_g2-s_f2)
    return Xf, Xg

def print_quality(chem1, chem2):
    Xf, Xg = quality(chem1, chem2)
    print('The quality of the saturated liquid {0} at state 2 is {1:8.3f}'.format(chem1[0], Xf))
    print('The quality of the saturated vapor {0} at state 2 is {1:8.3f}'.format(chem1[0], Xg))
    
def print_units(num_w_units):
    '''
    This function returns a formated str from a number with pint units.
    '''
    if num_w_units.magnitude < 0.1:
        value = print('{:~.3f}'.format(num_w_units))
    else:
        value = print('{:~.2f}'.format(num_w_units))
    return value

# Introduction
On June 21, 2019, at approximately 4:22 AM, a fire and explosion, see \cref{fig:fire}, occurred at the Girard Point Refinery owned by Philadelphia Energy Solutions \cite{Renshaw2019}.  One of the three explosions observed that day was from the V1 treater-feed-surge-drum in the refineries pretreatment unit.  The explosion propelled the largest piece (estimated: $41809.6lb\:(18964.5kg)$) of the drum approximately $2100ft\:(640m)$ from the blast seat, see \cref{fig:tank}.  It is hypothesized that a boiling-liquid expanding-vapor explosion (BLEVE) event provided the energy to generate the blast wave.  The Philadelphia Fire Department requested ATF estimate the blast overpressure generated when the tank exploded.  This paper is an engineering analysis to estimate the blast overpressure assuming a BLEVE occurred.  The analysis is based upon an adiabatic and isentropic energy analysis developed by the Center for Chemical Process Safety \cite{Safety2010}. 

![A view of the Philadelphia Energy Solutions Inc's oil refinery while on fire \cite{Maykuth2019}.](report_images/fig_refinery_fire.png){#fig:fire width=75% placement='H'}

In [ ]:
image1 = 'report_images/drum_in_flight.png'
image2 = 'report_images/drum_landing.png'
nb_setup.images_vconcat([image1, image2], width=340, gap=10)


# Background

## Refinery

The Girard Point Refinery is located in southwest Philadelphia, PA, on the Schuylkill River, see \cref{fig:map}.  The refinery produced approximately 335,000 bpd of gasoline, and was the largest on the East Coast \cite{AssociatedPress2019}.  The treater-feed-surge-drum, involved in the explosion, is part of the pretreatment process of alkylation used in the production of gasoline. 

![Map of Girard Point Refinery showing the blast seat location \cite{Duchneskie2019}.](report_images/fig_map_girard.jpeg){#fig:map width=75% placement='H'}

## Treater Feed Surge Tank

Alkylaition generally converts propylene $(C_3H_6)$, butylene $(C_4H_8)$, pentene $(C_5H_{10})$, and isobutane $(C_3H_{10})$ to alkane liquids such as isoheptane $(C_7H_{16})$ and isooctane $(C_8H_{16})$.  These alkylates are a highly valued component in the production of gasoline because of there high octane and low vapor pressure \cite{flowserve2000}.  The treater-feed-surge-drum (TFSD) was located between the fluid catalytic cracker and the alkylaition unit. The purpose of a surge drum is to stabilize fluctuations in the overall system flow rate.  The TFSD was part of the pretreatment process for alkylation, see \cref{fig:fig_process_flow}.  During pretreatment, also referred to as sweetening, sulfur compounds (hydrogen sulfide, thiophene and mercaptan) are removed to improve color, odor, and oxidation stability.

![Simple flow diagram of the sweetening and treating process where the TFSD was located at the time of the explosion. \cite{Temur2014,Malone2019}](report_images/fig_process_flow.jpeg){#fig:fig_process_flow width=75% placement='H'}

The TFSD tank measured 39'-8" in length, not including the heads, see \cref{tbl:tbl_tank} and \cref{fig:fig_tank} for construction details \cite{PES2019}. At the time of the explosion, the TFSD contained $20160gal\:(76.3m^3)$ of butane (50% by volume) and butene (40% by volume) and other lesser constituients, see \cref{tbl:tbl_chemicals_in_tank}.

Parameter             Value              Units
--------              --------           --------
Type                  horizontal         NA
Year Built            1972               NA
Construction Material A516 Type 70 Steel NA
Tank Wall Thickness   0.8125             in
Volume                372228             gal
Percent Filled        53.3               %
Mass                  74660              lb
Safety Valve Set      155                psig

Table: Treater-Feed-Surge-Drum Construction Parameters \cite{PES2019} {#tbl:tbl_tank align=rll widths=[0.25,0.25,0.25]}

![Diagram depicting the dimensions of the TFSD tank.  The tank was positioned 25 ft above ground level and estimated to contain 20160 gal (53.3% filled) of butane (50% by volume) and butene (40% by volume) at the time of the explosion.\cite{PES2019}](report_images/fig_tank.jpeg){#fig:fig_tank width=80% placement='H'}

Chemical  Percent by Volume
--------  --------
methane     0.01
ethylene    0.00
ethane      0.01
propane     0.90
propylene   0.10
isobutane  37.28
nbutane    12.81
butens     40.41
neopentane  0.00
isopentane  3.94
npentane    0.25
butadiene   0.33
benzene     0.00
C5 olefins  3.54
C6 sats     0.33
C7+         0.04

Table: Chemical Contents in the Treater-Feed-Surge-Drum Nearest to the Time of the Explosion \cite{PES2019} {#tbl:tbl_chemicals_in_tank align=rr widths=[0.2,0.2]}

For this analysis it will be assumed that the entire mixture is butane.  This assumption greatly simplifies the analysis, avoiding partial pressures and multiple energy calculations with different vapor qualities.  The assumption is reasonable because butane and butene make up more than 90% of the fluid volume and butane and butene are chemically similar, see \cref{tbl:tbl_butane_prop}.  The boiling temperatures for butene and butane are $20.66\: ^{\circ} F\:(-6.3\: ^{\circ} C)$ and $31.1\: ^{\circ} F\:(-1.0\: ^{\circ} C)$ respectively; both are well above the tank temperature when exposed to fire so a BLEVE assumptions is reasonable.






| Property                  | Butene      | Butane     |
|---------------------------|-------------|------------|
| Molecular Formula         | $C_4H_8$        | $C_4H_{10}$      |
| Molecular Weight          | $56.108\frac{g}{mol}$ | $58.12g\frac{g}{mol}$ |
| Boiling Point at 760mm Hg | $20.6^{\circ}F$ | $31.1^{\circ}F$     |
| Flash Point               | $-110.0^{\circ}F$ | $-76.0^{\circ}F$      |
| Density at 25$^{\circ} C$, 1 atm    | $0.588\frac{g}{cc}$  | $0.573\frac{g}{cc}$ |
| $\frac{C_P}{C_V}$ Ratio $\gamma$  |1.13    | 1.12                 |

Table: Chemical Properties of Butene and Butane \cite{PubChem-butane, PubChem-butene} {#tbl:tbl_butane_prop align=rrr widths=[0.3,0.3,0.3]}

## Pressure Relief Valve

The TFSD was fitted with a Consolidated (1906-30LC-1-CC-MS-31-RF-1) 3" x 4" pressure relief valve (PRV).  The relief pressure was set to $155 psig\:(1068.7kPa)$ and the relief temperature was set to $183.5^{\circ} F\:(84.3^{\circ}C)$.  The PRV was positioned on the top of the TRFD, see \cref{fig:fig_tank}.

## Recovered Drum Debris
Three major pieces of the TFSD were identified after the explosion.  They were thrown a maximum of $2100ft\:(640m)$, see \cref{fig:blast_debris}.  The largest piece (Large End Cap) was approximately $22ft\:(6.7m)$ in length, see \cref{fig:large_end_cap}, or a little more than half the original tank length.  The other end of the TFSD (Small End Cap) was recovered $1761ft\:(536m)$ from the blast seat.  It was approximately $5ft\:(1.5m)$ in length, see \cref{fig:small_end_cap}.  The piece thrown the shortest distance (fillet) was $819ft\:(249m)$ from the blast seat.  The fillet piece was heavily damaged and a photographic analysis of the length was not possible.  However, based on the original length of the tank and removing the large and small end cap lengths the fillet length is approximately $12ft\:(3.6m)$, see \cref{fig:fillet}.

![Map of Girard Point showing the blast seat and locations of the three main pieces of the TSFD (V-1).\cite{Malone2019a}](report_images/drum_debris_and_blast_seat_distances_small.png){#fig:blast_debris width=75% placement='H'}

![Photograph of the large end cap, estimated to be $22ft\:(6.7m)$ in length based on photographic analysis.  In this photograph the tank has been moved from it's original landing location.\cite{Malone2019a}](report_images/tank_length_estimate.png){#fig:large_end_cap width=75% placement='H'}

![Photograph of the small end cap, estimated to be $5ft\:(1.5m)$ in length based on photographic analysis.\cite{Malone2019a}](report_images/end_cap_03.jpg){#fig:small_end_cap width=75% placement='H'}

![Photograph of the fish fillet piece, calculated to be $12ft\:(3.6m)$ in length based on the total length of the drum less the lengths of the large and small end caps.\cite{Malone2019a}](report_images/1_fish_fillet2.jpg){#fig:fillet width=75% placement='H'}

The estimated mass of each piece of the TFSD is summarized in \cref{tbl:tbl_tank}.  The mass calculations are based on the refinery recorded mass of the vessel, $74660lb\:(33865kg)$, multiplied by the percent mass calculated using the following equations,

Ellipsoidal Head: $$V_{eh}=\frac{\pi D^2 (D/4)}{6}$$
Cylinder: $$V_c = \frac{\pi D^2 l}{4}$$

where the drum wall thickness is $0.83in\:(2.1cm)$, the tank diameter is $12ft\:(3.7m)$, and the material density for A516 steel is $7.8g/cc$\cite{Malone2019}.  The difference in calculated and refinery recorded mass is likely due to appurtenances attached to the tank not accounted for in the calculated mass.
 






TFSD ID               Cylinder (ft)      Calculated Debris Mass (lb)   Percent Mass Calculated         Refinery Recorded Mass (lb)  
--------              -------------      --------------------          -----------------------------   ---------------------
Large End Cap         22.0               31541.8                       0.56                            41809.6
Small End Cap          5.0               10078.5                       0.18                            13438.8
Fillet                12.0               15150.5                       0.26                            19411.6
Total                 39.0               56770.8                       1.00                            74660.0

Table: Estimated Treater-Feed-Surge-Drum Debris Mass \cite{PES2019} {#tbl:tbl_tank align=lrrrr widths=[0.2,0.1,0.1,0.15,0.15]}


## Boiling-Liquid Expanding-Vapor Explosion (BLEVE)

A BLEVE results from the sudden failure of a tank containing a compressed vapor (head space) and a super-heated liquid (a liquid heated above it's boiling point but without boiling). The magnitude of the blast depends on how super-heated the liquid was at failure.  As the level of super-heat rises, the portion of liquid that flash-boils rises, thus increasing the energy released.  Once containment failure occurs the energy is distributed into four forms:

1. Overpressure wave
2. Kinetic energy of fragments
3. Deformation and failure of the containment material
4. Heat transferred to environment

The distribution of the energy into the these four forms depends on the specifics of the explosion.  Planas-Cuchi et al. found that a *fragile* failure releases 80% of the energy into the blastwave, while a *ductile* failure releases 40% of the energy into the blastwave.  The remaining energy becomes kinetic energy of the fragments.  The heat transfer to the environment is relatively small \cite{Planas2004}.  In practice most pressure vessels are designed with materials that are ductile rather than brittle to avoid sudden and catastrophic brittle (fragile) failures \cite{Benac2016}.



In [ ]:
# Large End Cap Weight
dia = 12 * u.feet
thk = 0.06916667 * u.feet
lng = 22 * u.feet
den = 487 * u.lb/u.ft**3

hv = ellipsoidal_head(dia, thk)
cv = cylinder(lng, dia, thk)

wt_h = hv*den
wt_c = cv*den
wt_t = wt_c + wt_h
print_units(wt_t)

In [ ]:
# Small End Cap Weight
dia = 12 * u.feet
thk = 0.06916667 * u.feet
lng = 5 * u.feet
den = 487 * u.lb/u.ft**3

hv = ellipsoidal_head(dia, thk)
cv = cylinder(lng, dia, thk)

wt_h = hv*den
wt_c = cv*den
wt_t = wt_c + wt_h
print_units(wt_t)

In [ ]:
# Fillet Weight
dia = 12 * u.feet
thk = 0.06916667 * u.feet
lng = 12 * u.feet
den = 487 * u.lb/u.ft**3

cv = cylinder(lng, dia, thk)

wt_c = cv*den

print_units(wt_c)

In [ ]:
p_relief = 1068689.9 * u.Pa
p_atm = 101325.0 * u.Pa
p_1 = 1.21*(p_relief + p_atm)
p_2  = p_atm; #
V_1 = 140.8197 * u.m**3
Pct_f = 0.533 # percent fluid in the tank
Pct_g = 1 - Pct_f # percent vapor in the tank
R = 25* u.ft
a0 = 340 * u.m / u.s
pct_butene = 0.4041 # percent by volume
pct_butane = 0.5009 # percent by volume
vol_chem = 76.313904 * u.m**3
vol_butene = tot_vol * pct_butene
vol_butane = tot_vol * pct_butene

In [ ]:
p_relief_s = print('{:~.2f} ({:~.2f})'.format(p_relief.to(u.psi), p_relief.to(u.MPa)))
p_atm_s = print('{:~.2f} ({:~.2f})'.format(p_atm.to(u.psi), p_atm.to(u.MPa)))
p_1_s = print('{:~.2f} ({:~.2f})'.format(p_1.to(u.psi), p_1.to(u.MPa)))

In [ ]:
print_ie('1','butane',0)
print_ie('2','butane',1)
print_ie('1','propane',4)
print_ie('2','propane',5)

# Calculation of Air Blast from a BLEVE
There is a seven step process for calculating the air blast from a BLEVE:

1.  Data Collection

    a.  Vessel internal pressure, $p_1$
    b.  Ambient air pressure, $p_2$
    c.  Vessel volume
    d.  Ratio of specific heats of gas in the vessel
    e.  Distance from the center of the tank to the receptor
    f.  Shape of the vessel (spherical or cylindrical)
    h.  Speed of sound of air
    i.  Ratio of specific heats of air
    
2.  Calculate the Vessel-Gas Energy Using Brodes Equation \cite{Brode1959}
3.  Calculate the Vessel Burst Pressure Ratio $p_1/p_2$ and Select Blast Curves
4.  Calculate the Scaled Standoff Distance $\overline R$ of the Receptor \cite{Baker1973}
5.  Determine the Scaled Positive Overpressure and Impulse
6.  Adjust for Elevated Temperatures and Vessel Geometry
7.  Calculate Overpressure and Impulse

## Data Collection
For this analysis the following data will be used:

1.  Ambient air pressure, $14.7psi\:(0.101MPa)$
2.  Vessel volume, $372228gal\:(1409.0m^3)$
3.  Ratio of specific heats of butane (1.12)
4.  Distance from the center of the tank to the receptor, $25ft\:(7.62m)$
5.  Shape of the vessel is cylindrical $(L/D=3.3)$
6.  Speed of sound of air, $1115.49 ft/s\:(340m/s)$ 

## Pressure at State 1 (Pre-failure State)
The drum (tank) is assumed to fail at $1.21$ times the opening pressure of the pressure relief valve (PRV)\cite{Engineers2013}.  This pressure is based on the American Petroleum Institutes Standard 521 which, requires that pressure relief valves on pressure vessels achieve rated flow at 1.21 times the maximum allowable working pressure.  The PRV was set to $155psig\:(1.07Mpa)$ therefore, the absolute pressure at state 1 (failure state) is given by,

$$p_1 = 1.21\left(p_{PRV}+p_{atm}\right)$$
$$p_1 = 1.21\left(14.70+155.00\right)$$
$$p_1 = 205.33 psia\:(1.42\:MPa)$$

## Pressure at State 2 (Final Expanded State)
The pressure at state 2 (final expanded state) is standard atmospheric pressure or $14.7psi\:(0.101MPa)$.
## Internal Energy at States 1 and 2
Th internal energy $u$ can be used to estimate the energy released in an explosion.  With the gases in the saturated state and knowing the pressures at state 1 (explosion) and state 2 (atmospheric) we can use lookup tables determine the specific volume $v$, and the enthalpy $h$.  Combining these two properties with the pressure we can calculate the internal energy $u$ using,

$$h = u + pv$$

where $h$ is the enthalpy, $p$ is the pressure, and $v$ is the specific volume.  Therefore, solving for the internal energy $u$ we have,

$$u = h - pv$$

The internal energy at at state 1 for butane in the liquid state is,

$$u_{1f-butane} = h_{1f-butane} - (p_{1-butane})(v_{1f-butane})$$
$$u_{1f-butane} = 451.46 - (1415.72\:kPa)(0.002261\:m^3/kg)$$
$$u_{1f-butane} = 271.68\:kJ/kg$$

and for the gaseous butene we have,

$$u_{1g-butene} = h_{1g-butene} - (p_{1-butene})(v_{1g-butene})$$
$$u_{1g-butene} = 515.58 - (1937.60\:kPa)(0.0021\:m^3/kg)$$
$$u_{1g-butene} = 477.49\:kJ/kg$$

and similarly for butene at state 2,

$$u_{2f-butene} = -0.16\:kJ/kg$$
$$u_{2g-butene} = 354.35\:kJ/kg$$

and for states 1 and 2 for butane we have,

$$u_{1f-butane} = 498.49\:kJ/kg$$
$$u_{1g-butane} = 697.31\:kJ/kg$$
$$u_{2f-butane} = 198.70\:kJ/kg$$
$$u_{2g-butane} = 547.18\:kJ/kg$$

All the gas properties are summarized in \cref{tbl:thermo}

In [ ]:
#propane calculations for comparision example problem *** DELETE ***
p_relief = 1500000 #Pa
p_atm = 101325 #Pa 
p_p = 1.21*(p_relief + p_atm) # Pa absolute
propane_1 = chem_prop(p_p,'Propane','1')
propane_2 = chem_prop(p_2.magnitude,'Propane','2')
propane_dict = build_state_table(propane_1,propane_2)

# Philadelphia Refinery Calculations
# Butane
butane_1 = chem_prop(p_1.magnitude,'butane','1')
butane_2 = chem_prop(p_2.magnitude,'butane','2')
butane_dict = build_state_table(butane_1,butane_2)
# Butene
butene_1 = chem_prop(p_1.magnitude,'Butene','1')
butene_2 = chem_prop(p_2.magnitude,'Butene','2')
butene_dict = build_state_table(butene_1,butene_2)

# Combine all calculations into a Pandas Dataframe
pd = nb_setup.setup_pandas(escape_latex=False)
df2 = pd.DataFrame(butene_dict)
df1 = pd.DataFrame(butane_dict)
df3 = pd.DataFrame(propane_dict)
df = pd.concat([df1, df2, df3])
df.set_index('State', inplace=True)
pd.options.display.float_format = '{:.2E}'.format
df

In [ ]:
print_quality(propane_1, propane_2)
print_quality(butene_1, butene_2)
print_quality(butane_1, butane_2)

## Internal Energy at State 2
When the drum breaks and the butane and butene at state 1 expands to state 2 (atmospheric pressure) some of the liquid vaporizes and some of the gases condense. Therefore, unlike at the saturated state 1, there is both vapor and fluid present at state 2.  We can use the quality of the vapor and liquid to calculate the internal energy at state 2.  The vapor quality $(\chi)$, can be calculated from,

$$\chi = \frac{s - s_f}{s_{g2}-s_{f2}}$$

where $s$ is the entropy of the butane or butene.  For the saturated liquid expanding from state 1 to state 2 we have,

$$\chi_f = \frac{s_{f1} - s_{f2}}{s_{g2}-s_{f2}}$$

$$\chi_f = \frac{1.503188 - 0.607045}{2.449144-0.607045}$$

$$\chi_f = 0.4865$$

and for the vapor at state 2,

$$\chi_g = \frac{s_{g2} - s_{g1}}{s_{g2}-s_{f2}}$$

$$\chi_g= \frac{2.449144 - 2.325970}{2.449144-0.607045}$$

$$\chi_g = 0.06687$$

We can then calculate the internal energy at state 2 using,

$$u_{2-fluid} = (1-\chi_f)u_{f2} + \chi_f u_{g2}$$
$$u_{2-vapor} = (1-\chi_g)u_{g2} + \chi_g u_{f2}$$

$$u_{2-fluid} = (1-0.4865)100.1818 + (0.4865)(484.0111)$$
$$u_{2-vapor} = (1-0.0668)484.0111 + (0.0668)(100.1818)$$

$$u_{2-fluid} = 286.9068\:kJ/kg$$
$$u_{2-vapor} = 458.3459\:kJ/kg$$

In [ ]:
e_exf = u_f1 - u_2_fluid
e_exg = u_g1 - u_2_vapor

print("The specific work done by the saturated liquid in expansion from state 1 to state 2 is %8.2f kJ/kg" % (e_exf))
print("The specific work done by the saturated vapor in expansion from state 1 to state 2 is %8.2f kJ/kg" % (e_exg))

# The Specific Work
The work that the expanding vapor and fluid can perform is the difference between the initial (1) and final (2) states,

$$e_{ex} = u_1 - u_2$$

for the saturated fluid we have,

$$e_{exf} = u_{f1} - u_{2-fluid}$$
$$e_{exf} = 350.15 - 286.91$$
$$e_{exf} = 63.25\:kJ/kg$$

and for the vapor,

$$e_{exg} = u_{g1} - u_{2-vapor}$$
$$e_{exg} = 581.68 - 458.35$$
$$e_{exg} = 123.34\:kJ/kg$$

In [ ]:
m_f1 = Pct_f*V_1/v_f1 # mass of fluid at state 1 (kg)
m_g1 = Pct_g*V_1/v_g1 # mass of gas at state 1 (kg)
print("The mass of the fluid at state 1 is %8.2f kg" % (m_f1))
print("The mass of the vapor at state 1 is %8.2f kg" % (m_g1))
E_exf = 2*e_exf*m_f1 # explosion energy of fluid on the ground (factor of 2) at state 1 (kJ)
E_exg = 2*e_exg*m_g1 # explosion energy of gas on the ground at state 1 (kJ)
print("The explosion energy of the fluid at state 1 is %8.2f kJ." % (E_exf))
print("The explosion energy of the vapor at state 1 is %8.2f kJ." % (E_exg))
E_extot = E_exf+E_exg # total explosion energy on the ground at state 1
print("The total energy of the surface explosion is Ex_tot = %8.2f kJ." % (E_extot))

# The Explosion Energy
The explosion energy is calcu

In [ ]:
R_bar = R*(101325/1156.1e6)**(1/3)
print("The non-dimensional range of the receptor is %8.2f" % (R_bar))
print("The non-dimensional tank pressure p/po is %8.2f" % (p_1/p_atm))

In [ ]:
pr5 = pd.read_csv('bleve_overpressure_curves/p1_p0_5.csv'); pr5['z'] = r'$p/p_0=5.0$'; pr5['w'] = 5.0
pr10 = pd.read_csv('bleve_overpressure_curves/p1_p0_10.csv'); pr10['z'] = r'$p/p_0=10.0$'; pr10['w'] = 10.0
pr20 = pd.read_csv('bleve_overpressure_curves/p1_p0_20.csv'); pr20['z'] = r'$p/p_0=20.0$'; pr20['w'] = 20.0
pr50 = pd.read_csv('bleve_overpressure_curves/p1_p0_50.csv'); pr50['z'] = r'$p/p_0=50.0$'; pr50['w'] = 50.0
pr100 = pd.read_csv('bleve_overpressure_curves/p1_p0_100.csv'); pr100['z'] = r'$p/p_0=100.0$'; pr100['w'] = 100.0
pr200 = pd.read_csv('bleve_overpressure_curves/p1_p0_200.csv'); pr200['z'] = r'$p/p_0=200.0$'; pr200['w'] = 200.0
pr500 = pd.read_csv('bleve_overpressure_curves/p1_p0_500.csv'); pr500['z'] = r'$p/p_0=500.0$'; pr500['w'] = 500.0
pr1000 = pd.read_csv('bleve_overpressure_curves/p1_p0_1000.csv'); pr1000['z'] = r'$p/p_0=1000.0$'; pr1000['w'] = 1000.0
frame = [pr5, pr10, pr20, pr50, pr100, pr200, pr500, pr1000]
df_pr = pd.concat(frame, sort=True)
df_pr.head()

In [ ]:
#on a grid in [2, 18]x[0, 10] which was the axes of the
#original paper. The imaginary numbers 200j will break
#the grid into 200 equal spaces.

grid_x, grid_y = np.mgrid[0:10:200j, 0:1000:200j]

#we know the x,y,z values from the original plot image 

points = df_pr[['R','P']].values
values = df_pr['w'].values

points_df_pr = df_pr[['R','w']].values
values_df_pr = df_pr['P'].values

#The cubic method of griddata was used for interpolation.  Grid_x and Grid_y
#can be either a matrix of points or a single pair.
grid_z2_pr = griddata(points_df, values_df, (grid_x, grid_y), method='cubic')

sd_p_vert = 20
sd_r_horz = 4.4
pr = griddata(points_df_pr, values_df_pr, (sd_r_horz, sd_p_vert), method='cubic')
print("The non-dimensional incident overpressure is %8.3f" % (pr))

In [ ]:
# Get an empty figure
fig1 = plt.figure(figsize = [w,h])

# Get the axes instance at 1st location in 1x1 grid
ax = fig1.add_subplot(1,1,1)

# Generate the plot
grpd = df_pr.groupby('w').apply(lambda x: x.sort_values('z')) # Group by 'w' then sort by 'z'
for name in grpd['z'].unique(): # Iterate through grouped values of z and plot
    plt.loglog(df_pr[df_pr.z==name].R.values, df_pr[df_pr.z==name].P.values, label = name);
plt.plot(sd_r_horz,pr,'rp', label = '$Nondimensional\:Overpressure$');
plt.legend();
plt.axhline(y=pr, color='r', linestyle='-')
plt.axvline(x=sd_r_horz, color='r', linestyle='-')
#plt.text(sd_r_horz, pv/2, '$\left({{{}}},{{{}}}\right)$'.format([sd_r_horz,pv]), ha='right', va='center')

# Set labels for x and y axis
ax.set_xlabel('$R/(E/p_0)^{1/3}$');
ax.set_ylabel('$(P-p_0)/p_0$');
ax.grid(True, which='both')

# Display the figure
plt.show();

In [ ]:
ir5 = pd.read_csv('bleve_impulse_curves/ip1_p0_5.csv'); ir5['z'] = r'$p/p_0=5.0$'; ir5['w'] = 5.0
ir10 = pd.read_csv('bleve_impulse_curves/ip1_p0_10.csv'); ir10['z'] = r'$p/p_0=10.0$'; ir10['w'] = 10.0
ir20 = pd.read_csv('bleve_impulse_curves/ip1_p0_20.csv'); ir20['z'] = r'$p/p_0=20.0$'; ir20['w'] = 20.0
ir50 = pd.read_csv('bleve_impulse_curves/ip1_p0_50.csv'); ir50['z'] = r'$p/p_0=50.0$'; ir50['w'] = 50.0
ir100 = pd.read_csv('bleve_impulse_curves/ip1_p0_100.csv'); ir100['z'] = r'$p/p_0=100.0$'; ir100['w'] = 100.0
ir200 = pd.read_csv('bleve_impulse_curves/ip1_p0_200.csv'); ir200['z'] = r'$p/p_0=200.0$'; ir200['w'] = 200.0
ir500 = pd.read_csv('bleve_impulse_curves/ip1_p0_500.csv'); ir500['z'] = r'$p/p_0=500.0$'; ir500['w'] = 500.0
ir1000 = pd.read_csv('bleve_impulse_curves/ip1_p0_1000.csv'); ir1000['z'] = r'$p/p_0=1000.0$'; ir1000['w'] = 1000.0
frame = [ir5, ir10, ir20, ir50, ir100, ir200, ir500, ir1000]
df_ir = pd.concat(frame, sort=True)
df_ir.head()

In [ ]:
#on a grid in [2, 18]x[0, 10] which was the axes of the
#original paper. The imaginary numbers 200j will break
#the grid into 200 equal spaces.

grid_x, grid_y = np.mgrid[0:10:200j, 0:1000:200j]

#we know the x,y,z values from the original plot image 

points = df_ir[['R','I']].values
values = df_ir['w'].values

points_df_ir = df_ir[['R','w']].values
values_df_ir = df_ir['I'].values

#The cubic method of griddata was used for interpolation.  Grid_x and Grid_y
#can be either a matrix of points or a single pair.
grid_z2_ir = griddata(points_df_ir, values_df_ir, (grid_x, grid_y), method='cubic')

sd_p_vert = 20
sd_r_horz = 4.4
ir = griddata(points_df_ir, values_df_ir, (sd_r_horz, sd_p_vert), method='cubic')
print("The non-dimensional incident impulse is %8.3f" % (ir))

In [ ]:
# Get an empty figure
fig2 = plt.figure(figsize = [w,h])

# Get the axes instance at 1st location in 1x1 grid
ax = fig2.add_subplot(1,1,1)

# Generate the plot
grpd = df_ir.groupby('w').apply(lambda x: x.sort_values('z')) # Group by 'w' then sort by 'z'
for name in grpd['z'].unique(): # Iterate through grouped values of z and plot
    plt.loglog(df_ir[df_ir.z==name].R.values, df_ir[df_ir.z==name].I.values, label = name);
plt.plot(sd_r_horz,ir,'rp', label = '$Nondimensional\:Overpressure$');
plt.legend();
plt.axhline(y=ir, color='r', linestyle='-')
plt.axvline(x=sd_r_horz, color='r', linestyle='-')

# Set labels for x and y axis
ax.set_xlabel('$R/(E/p_0)^{1/3}$');
ax.set_ylabel('$i\:a_0/(E^{1/3}p_0^{2/3})$');
ax.grid(True, which='both')

# Display the figure
plt.show();

In [ ]:
p_incident = pr*p_atm
print("The incident overpressure is %8.2f Pa (%8.2f psi)" % (p_incident, p_incident*0.000145038))

In [ ]:
i_s = (ir*((p_atm)**(2/3))*(E_extot*1000)**(1/3))/a0
print("The incident impulse is %8.2f Pa-s (%8.2f psi-s)" % (i_s, i_s*0.000145038))

## Nondimensional Side-on Pressure and Impulse
The nondimenstional side-on pressure can be calculated from Figure X and gives a $\bar{P_s} = 0.064$ for an $\bar{R}=4.4$ and a $p/p_0 = 19.12$.  The nondimensional side-on impulse can be calculated from Figure Y and gives a $\bar{i_s} = 0.013$ for an $\bar{R}=4.4$ and a $p/p_0 = 19.12$.

The side-on pressure and impulse can be calculated from the following:

$$\bar{P_s} - p_0 = (0.064)(101.325\:kPa) = 6.5\:kPa$$
$$i_s = \frac{(0.013)(101325\:kPa)^{2/3}(1160.9E6\:J)^{1/3}}{340\:m/s}= 86.31\:Pa {\text -} s$$
      